In [1]:
%%bash
pip freeze | grep tensor


tensor2tensor==1.9.0
tensorboard==1.10.0
tensorflow-gpu==1.10.1


You are using pip version 18.0, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [1]:
%%bash
rm -rf big_en_hi_small
mkdir -p big_en_hi_small/trainer

In [3]:
%pwd


'/home/phil'

In [2]:
%%writefile big_en_hi_small/trainer/__init__.py
from . import problem

Writing big_en_hi_small/trainer/__init__.py


In [3]:
%%writefile big_en_hi_small/setup.py
from setuptools import find_packages
from setuptools import setup

REQUIRED_PACKAGES = [
  'tensor2tensor'
]

setup(
    name='big_en_hi_small',
    version='0.1',
    author = 'phil',
    author_email = 'phildani7@nith.ac.in',
    install_requires=REQUIRED_PACKAGES,
    packages=find_packages(),
    include_package_data=True,
    description='big_en_hi_small Translation',
    requires=[]
)

Writing big_en_hi_small/setup.py


In [4]:
!touch big_en_hi_small/__init__.py

In [5]:
!find big_en_hi_small

big_en_hi_small
big_en_hi_small/__init__.py
big_en_hi_small/setup.py
big_en_hi_small/trainer
big_en_hi_small/trainer/__init__.py


In [6]:
%%writefile big_en_hi_small/trainer/problem.py
import os
import tensorflow as tf
from tensor2tensor.utils import registry
from tensor2tensor.models import transformer
from tensor2tensor.data_generators import problem
from tensor2tensor.data_generators import text_encoder
from tensor2tensor.data_generators import text_problems
from tensor2tensor.data_generators import generator_utils


@registry.register_problem
class Big_en_hi_small(text_problems.Text2TextProblem):
  """Predict next line of poetry from the last line. From Gutenberg texts."""

  @property
  def approx_vocab_size(self):
    return 2**13  # ~8k      edited by phildani7

  @property
  def is_generate_per_split(self):
    # generate_data will NOT shard the data into TRAIN and EVAL for us.
    return False

  @property
  def dataset_splits(self):
    """Splits of data to produce and number of output shards for each."""
    # 10% evaluation data
    return [{
        "split": problem.DatasetSplit.TRAIN,
        "shards": 90,
    }, {
        "split": problem.DatasetSplit.EVAL,
        "shards": 10,
    }]

  def generate_samples(self, data_dir, tmp_dir, dataset_split):
    with open('/home/phil/big_en_hi_small/trainer/input.txt', 'r') as inp, open('/home/phil/big_en_hi_small/trainer/output.txt', 'r') as out:
        for en_line, hi_line in zip(inp, out):
            yield {
                    "inputs": en_line,
                    "targets": hi_line
            }       


# Smaller than the typical translate model, and with more regularization
#@registry.register_hparams
#def transformer_big_en_hi_small():
#  hparams=transformer.transformer_base()
#  hparams.optimizer_adam_epsilon = 1e-9
#  hparams.learning_rate_decay_scheme = "noam"
#  hparams.learning_rate = 0.1
#  hparams.learning_rate_warmup_steps = 4000
#  hparams.initializer_gain = 1.0
#  #hparams.num_hidden_layers = 6
#  hparams.initializer = "uniform_unit_scaling"
#  hparams.weight_decay = 0.0
#  hparams.optimizer_adam_beta1 = 0.9
#  hparams.optimizer_adam_beta2 = 0.98
#  hparams.num_sampled_classes = 0
#  hparams.label_smoothing = 0.1
#  hparams.shared_embedding_and_softmax_weights = int(True)








 
#  hparams.num_hidden_layers = 2
#  hparams.hidden_size = 128
#  hparams.filter_size = 512
#  hparams.num_heads = 4
#  hparams.attention_dropout = 0.6
#  hparams.layer_prepostprocess_dropout = 0.6
#  hparams.learning_rate = 0.05
#  return hparams

# hyperparameter tuning ranges
#@registry.register_ranged_hparams
#def transformer_big_en_hi_small_range(rhp):
#  rhp.set_float("learning_rate", 0.05, 0.25, scale=rhp.LOG_SCALE)
#  rhp.set_int("num_hidden_layers", 2, 4)
#  rhp.set_discrete("hidden_size", [128, 256, 512])
#  rhp.set_float("attention_dropout", 0.4, 0.7)

Writing big_en_hi_small/trainer/problem.py


In [7]:
%%bash
DATA_DIR=./t2t_data
TMP_DIR=$DATA_DIR
rm -rf $DATA_DIR $TMP_DIR
mkdir -p $DATA_DIR $TMP_DIR
# Generate data
t2t-datagen \
  --t2t_usr_dir=/home/phil/big_en_hi_small/trainer \
  --problem=big_en_hi_small \
  --data_dir=$DATA_DIR \
  --tmp_dir=$TMP_DIR

/home/phil/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
INFO:tensorflow:Importing user module trainer from path /home/phil/big_en_hi_small
INFO:tensorflow:Generating problems:
    big:
      * big_en_hi_small
INFO:tensorflow:Generating data for big_en_hi_small.
INFO:tensorflow:Generating vocab file: ./t2t_data/vocab.big_en_hi_small.8192.subwords
INFO:tensorflow:Trying min_count 500
INFO:tensorflow:Iteration 0
INFO:tensorflow:vocab_size = 17260
INFO:tensorflow:Iteration 1
INFO:tensorflow:vocab_size = 8070
INFO:tensorflow:Iteration 2
INFO:tensorflow:vocab_size = 8298
INFO:tensorflow:Iteration 3
INFO:tensorflow:vocab_size = 8267
INFO:tensorflow:Generating case 0.
INFO:tensorflow:Generating case 100000.
INFO:tensorflow:Generating case 2

In [18]:
%%bash
DATA_DIR=/home/phil/t2t_data
OUTDIR=/home/phil/big_en_hi_small/trained_model
t2t_tmp=$DATA_DIR
#rm -rf $OUTDIR
t2t-trainer \
  --data_dir=$DATA_DIR \
  --t2t_usr_dir=./big_en_hi_small/trainer \
  --problem=big_en_hi_small \
  --model=transformer \
  --hparams_set=transformer_base \
  --output_dir=$OUTDIR \
  --worker_gpu=2 \
  --train_steps=10000000 
  #--sync=True
  #--eval_steps=1000 \ #--sync=True \
  #--log_step_count_steps=1000


Process is terminated.


In [34]:
%%bash
DATA_DIR=/home/phil/t2t_data
OUTDIR=/home/phil/trained_model
DECODE_FILE=/home/phil/big_en_hi_small/my.txt

PROBLEM=big_en_hi_small
MODEL=transformer
HPARAMS=transformer_base



BEAM_SIZE=4
ALPHA=0.6

t2t-decoder \
  --data_dir=$DATA_DIR \
  --problem=big_en_hi_small \
  --model=transformer \
  --hparams_set=transformer_base \
  --output_dir=/home/phil/trained_model \
  --t2t_usr_dir=/home/phil/big_en_hi_small/trainer \
  --decode_hparams="beam_size=$BEAM_SIZE,alpha=$ALPHA" \
  --decode_from_file=$DECODE_FILE

INFO:tensorflow:Importing user module trainer from path /home/phil/bible
Instructions for updating:
When switching to tf.estimator.Estimator, use tf.estimator.RunConfig instead.
INFO:tensorflow:schedule=continuous_train_and_eval
INFO:tensorflow:worker_gpu=1
INFO:tensorflow:sync=False
INFO:tensorflow:datashard_devices: ['gpu:0']
INFO:tensorflow:caching_devices: None
INFO:tensorflow:ps_devices: ['gpu:0']
INFO:tensorflow:Using config: {'_task_type': None, '_task_id': 0, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fa761b95ef0>, '_master': '', '_num_ps_replicas': 0, '_num_worker_replicas': 0, '_environment': 'local', '_is_chief': True, '_evaluation_master': '', '_train_distribute': None, '_device_fn': None, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_secs': None, '_log_step_count_steps': 100, '_session_config': gpu_options {
  per_process_gpu_memory_fractio